<a align="center" href="https://ultralytics.com/hub" target="_blank">
<img width="1024", src="https://github.com/ultralytics/assets/raw/main/im/ultralytics-hub.png"></a>

<div align="center">

[中文](https://docs.ultralytics.com/zh/hub/) | [한국어](https://docs.ultralytics.com/ko/hub/) | [日本語](https://docs.ultralytics.com/ja/hub/) | [Русский](https://docs.ultralytics.com/ru/hub/) | [Deutsch](https://docs.ultralytics.com/de/hub/) | [Français](https://docs.ultralytics.com/fr/hub/) | [Español](https://docs.ultralytics.com/es/hub/) | [Português](https://docs.ultralytics.com/pt/hub/) | [Türkçe](https://docs.ultralytics.com/tr/hub/) | [Tiếng Việt](https://docs.ultralytics.com/vi/hub/) | [العربية](https://docs.ultralytics.com/ar/hub/)

  <a href="https://github.com/ultralytics/hub/actions/workflows/ci.yaml"><img src="https://github.com/ultralytics/hub/actions/workflows/ci.yaml/badge.svg" alt="CI CPU"></a>
  <a href="https://colab.research.google.com/github/ultralytics/hub/blob/main/hub.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>

Welcome to the [Ultralytics](https://ultralytics.com/) HUB notebook!

This notebook allows you to train Ultralytics [YOLO](https://github.com/ultralytics/ultralytics) 🚀 models using [HUB](https://hub.ultralytics.com/). Please browse the HUB <a href="https://docs.ultralytics.com/hub/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/hub/issues/new/choose">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!
</div>

#环境初始化

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
%pip install ultralytics  # install
from ultralytics import YOLO, checks, hub
from ultralytics.utils.files import WorkingDirectory
from ultralytics.utils.downloads import download
import shutil
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
checks()  # checks

current = os.path.dirname(os.path.abspath('__file__'))
parent = Path(current+'/codes')
if  parent.exists():
  shutil.rmtree(parent)
os.makedirs(parent)
download('https://github.com/zh79325/jxh-ai-server/archive/refs/heads/main.zip',dir=parent)



Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 32.4/235.7 GB disk)


184MB [00:16, 12.0MB/s]
Unzipping /content/codes/main.zip to /content/codes/jxh-ai-server-main...: 100%|██████████| 49/49 [00:01<00:00, 32.99file/s]


# 下载人工打标数据集


In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas as pd
from glob import glob
import shutil
import yaml
import json
import urllib
from ultralytics.utils.downloads import download
checks()  # checks
dataset_url='https://aiimg.obs.cn-east-3.myhuaweicloud.com/train/dataset/project-4-at-2024-11-13-00-59-ce04fdff.zip'
current = os.path.dirname(os.path.abspath('__file__'))
root = current+'/codes/jxh-ai-server-main/jxh-ai-models'
datasset_folder=Path(root+'/ls-export')
folder=root+'/dataset-download'
datasset_download_folder =Path(folder)
if  datasset_download_folder.exists():
  shutil.rmtree(datasset_download_folder)
os.makedirs(datasset_download_folder)
download(dataset_url,datasset_download_folder)
subfolders= [f.path for f in os.scandir(folder) if f.is_dir()]
folder=subfolders[0]
datasets = root+'/yolo_datasets'
model_yaml = os.path.abspath(f'{datasets}/data.yaml')
print('\n\n\n==========\n\n')
print('datasset_download_folder=>',folder,end='\n')
txts = glob(folder + '/**/*.txt')
images = glob(folder + '/**/*.jpg')

# Create DataFrame
df = pd.DataFrame({'txt': txts, 'image': images})

if os.path.exists(datasets):
    shutil.rmtree(datasets)
# Shuffle and split data
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
all_train = len(df) < 20
train_size = int(0.8 * len(df))
if all_train:
    train_size = len(df)
train_df = df.iloc[:train_size]
test_df = []
val_df = []
if all_train:
    test_df = train_df
    val_df = train_df
else:
    test_df = df.iloc[train_size:]
    val_df = test_df.sample(frac=0.5)
    test_df = test_df.drop(val_df.index)

print('building data set')
# Create directories
for split in ['train', 'test', 'val']:
    os.makedirs(f'{datasets}/{split}/images', exist_ok=True)
    os.makedirs(f'{datasets}/{split}/labels', exist_ok=True)

# Copy files to respective directories
for split, split_df in [('train', train_df), ('test', test_df), ('val', val_df)]:
    for _, row in split_df.iterrows():
        shutil.copy(row['image'], f'{datasets}/{split}/images')
        shutil.copy(row['txt'], f'{datasets}/{split}/labels')

with open(f'{folder}/notes.json', 'r') as file:
    notes = json.load(file)

labelList = []
for label in notes['categories']:
    name = urllib.parse.unquote(label['name'])
    labelList.append(name)

print("parse yaml success labels =>", labelList)

data = dict(
    train='train/images',
    val='val/images',
    test='test/images',
    nc=len(labelList),
    names=labelList
)
with open(model_yaml, 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

print('finish model_yaml in=>', model_yaml)


Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 32.7/235.7 GB disk)


100%|██████████| 2.59M/2.59M [00:00<00:00, 5.35MB/s]
Unzipping /content/codes/jxh-ai-server-main/jxh-ai-models/dataset-download/project-4-at-2024-11-13-00-59-ce04fdff.zip to /content/codes/jxh-ai-server-main/jxh-ai-models/dataset-download/project-4-at-2024-11-13-00-59-ce04fdff...: 100%|██████████| 10/10 [00:00<00:00, 940.87file/s]






datasset_download_folder=> /content/codes/jxh-ai-server-main/jxh-ai-models/dataset-download/project-4-at-2024-11-13-00-59-ce04fdff
building data set
parse yaml success labels => ['五粮液', '剑南春', '劲酒', '底左1酒', '底左2酒', '底左3酒', '底左4酒', '底左5酒', '底左6酒', '底左7酒', '底左8酒', '红枸杞酒', '茅台王子', '西风十五年', '郎酒']
finish model_yaml in=> /content/codes/jxh-ai-server-main/jxh-ai-models/yolo_datasets/data.yaml


# 模型训练

In [3]:
import os
import shutil
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

from ultralytics import YOLO

mpl_data_path = mpl.get_data_path()
mpl_fonts_folder = mpl_data_path + '/fonts/ttf'
print("get_data_path=>", mpl.get_data_path())
if not os.path.exists(mpl_fonts_folder + '/SimHei.ttf'):
    shutil.copyfile(root + '/SimHei.ttf', mpl_fonts_folder + '/SimHei.ttf')
fontManager.addfont(mpl_fonts_folder + '/SimHei.ttf')
plt.rcParams["font.family"] = "SimHei"
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号


init_model=root+'/models/input/yolo11l.pt'
print('load init_model from =>',init_model)
print('load model_yaml from =>',model_yaml)
model = YOLO(init_model)  # build a new model from YAML
results = model.train(data=model_yaml, epochs=100, imgsz=1440)

get_data_path=> /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data
load init_model from => /content/codes/jxh-ai-server-main/jxh-ai-models/models/input/yolo11l.pt
load model_yaml from => /content/codes/jxh-ai-server-main/jxh-ai-models/yolo_datasets/data.yaml
Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/codes/jxh-ai-server-main/jxh-ai-models/models/input/yolo11l.pt, data=/content/codes/jxh-ai-server-main/jxh-ai-models/yolo_datasets/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1440, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val,

100%|██████████| 22.2M/22.2M [00:00<00:00, 39.3MB/s]


Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 358MB/s]


AMP: checks passed ✅


...

                   all          3        262      0.895      0.907      0.958      0.649

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100        11G      1.401      0.803      1.108        256       1440: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.32it/s]


                   all          3        262      0.886      0.907      0.957      0.652

100 epochs completed in 0.082 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11l summary (fused): 464 layers, 25,290,877 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.77it/s]


                   all          3        262      0.889      0.907      0.957      0.652
                   五粮液          3          6          1          1      0.995      0.865
                   剑南春          3          7       0.76          1      0.978      0.833
                    劲酒          3         15      0.963      0.933      0.991      0.744
                  底左1酒          3         15      0.968      0.867      0.958      0.563
                  底左2酒          3         15      0.713          1      0.943      0.532
                  底左3酒          3         27       0.89          1      0.963      0.562
                  底左4酒          3         28      0.876      0.893       0.97      0.647
                  底左5酒          3         33      0.901       0.97      0.925      0.508
                  底左6酒          3         19      0.894      0.447      0.917      0.604
                  底左7酒          2         15      0.877      0.867      0.928      0.545
                  底左8

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
       print(f"GPU index: {torch.cuda.current_device()}")

if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")
        for i in range(num_gpus):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


Using device: cuda
GPU index: 0
Number of GPUs available: 1
GPU 0: Tesla T4
